In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard
# from data import trainGenerator, testGenerator, saveResult, testGenerator2
import keras.backend as K
import os, cv2
import numpy as np
import skimage.io as io
import skimage.transform as trans
import tensorflow as tf
import shutil
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from keras.models import Model

In [ ]:
import os
import shutil
# Directory containing the training data
source_directory = '../DeepGlobe_road_extraction/train'   

# Directory where PNG images will be moved
satelliteDir = "satellite_images/"
maskedDir = "masked_images/" 

# Create the subdirectory if it doesn't exist
if not os.path.exists(satelliteDir):
    os.makedirs(satelliteDir)
    
if not os.path.exists(maskedDir):
    os.makedirs(maskedDir)

# List all files in the directory
all_files = os.listdir(source_directory)

# Filter and move PNG files to the subdirectory
for file in all_files:
    if file.endswith('.png'):
        # Full path of the source file
        source_path = os.path.join(source_directory, file)
        
        # Full path of the destination file
        destination_path = os.path.join(maskedDir, file)
        
        # Move the file to the subdirectory
        shutil.copy(source_path, destination_path)
    elif file.endswith('.jpg'):
        # Full path of the source file
        source_path = os.path.join(source_directory, file)
        
        # Full path of the destination file
        destination_path = os.path.join(satelliteDir, file)
        
        # Move the file to the subdirectory
        shutil.copy(source_path, destination_path)

In [11]:
sourceDir_sat="./satellite_images/"
sourceDir_mask="./masked_images/"
trainDir_sat="./trainData/sat/"
valDir_sat="./valData/sat/"
trainDir_mask="./trainData/mask/"
valDir_mask="./valData/mask/"
#taking 600 images to train
if not os.path.exists(trainDir_sat):
    os.makedirs(trainDir_sat)

if not os.path.exists(valDir_sat):
    os.makedirs(valDir_sat)

if not os.path.exists(trainDir_mask):
    os.makedirs(trainDir_mask)

if not os.path.exists(valDir_mask):
    os.makedirs(valDir_mask)


count=0
#train data (satellite)
for filename in os.listdir(sourceDir_sat):
    if(count>=600): 
        break
    if filename.lower().endswith('.jpg'):
        source_file = os.path.join(sourceDir_sat, filename)
        destination_file = os.path.join(trainDir_sat, filename)
        
        # Move the file
        shutil.move(source_file, destination_file)
        count+=1

count=0
#train data (masked)
for filename in os.listdir(sourceDir_mask):
    if(count>=600):
        break
    if filename.lower().endswith('.png'):
        source_file = os.path.join(sourceDir_mask, filename)
        destination_file = os.path.join(trainDir_mask, filename)
        
        # Move the file
        shutil.move(source_file, destination_file)
        count+=1

count=0
#val data(satellite)
for filename in os.listdir(sourceDir_sat):
    if(count>=50):
        break
    if filename.lower().endswith('.jpg'):
        source_file = os.path.join(sourceDir_sat, filename)
        destination_file = os.path.join(valDir_sat, filename)
        
        # Move the file
        shutil.move(source_file, destination_file)
        count+=1

count=0
#val data (mask)
for filename in os.listdir(sourceDir_mask):
    if(count>=50):
        break
    if filename.lower().endswith('.png'):
        source_file = os.path.join(sourceDir_mask, filename)
        destination_file = os.path.join(valDir_mask, filename)
        
        # Move the file
        shutil.move(source_file, destination_file)
        count+=1

In [ ]:
# Code to crop images to specific size

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def crop_image(src, save_path):
    TEST_SET = os.listdir(src)
    img_h = 256
    img_w = 256
    stride = img_h-40
    for n in range(len(TEST_SET)):
        image_name = TEST_SET[n]
        path1 = image_name[0:-7]+'mask.png'  #rename mask
        # load the image
        # image = cv2.imread(os.path.join(src,image_name), cv2.IMREAD_UNCHANGED)
        # image = cv2.imread(os.path.join(src,image_name))
        print(os.path.join(src,image_name))
        image = io.imread(os.path.join(src,image_name))
        
        print(image.shape)
        h, w, _ = image.shape
        # h, w = image.shape

        num = 0;
        #image = img_to_array(image)
        # padding_img = (padding_img - np.min(padding_img)) / (np.max(padding_img) - np.min(padding_img))

        print('[{}/{}], croping:{}'.format(n+1, len(TEST_SET), image_name))

        #mask_whole = np.zeros((h, w, 1), dtype=np.uint8)
        #temp = np.zeros((img_h, img_h), dtype=np.uint8)

        for i in range(0, (h // stride)+1):
            for j in range(0, (w // stride)+1):
                h_begin = i * stride
                w_begin = j * stride
                
                if h_begin + img_h > h:
                    h_begin = h_begin - (h_begin + img_h - h)
                
                if w_begin + img_w > w:
                    w_begin = w_begin - (w_begin + img_w - w)
                
                crop = image[h_begin:h_begin + img_h, w_begin:w_begin + img_w] 
                if num <= 9:
                    #path1 = image_name[0:-4]+'0'+ str(num)+'.jpg'
                    path1 = image_name[0:-4]+'0'+ str(num)+'.png'
                else:
                    #path1 = image_name[0:-4]+str(num)+'.jpg'
                    path1 = image_name[0:-4]+str(num)+'.png'
                
                #io.imsave(save_path + path1, crop)
                cv2.imwrite(save_path + path1, crop)
                num = num + 1
        print('Done!')

In [ ]:
# Cropping Train Images
trainDir_sat="./trainData/sat/"
valDir_sat="./valData/sat/"
trainDir_mask="./trainData/mask/"
valDir_mask="./valData/mask/"

cropsatDir_train = "./trainData/cropped_sat/"
cropmaskDir_train = "./trainData/cropped_mask/" 

cropsatDir_val = "./valData/cropped_sat/"
cropmaskDir_val = "./valData/cropped_mask/" 

if not os.path.exists(cropsatDir_train):
    os.makedirs(cropsatDir_train)
    
if not os.path.exists(cropmaskDir_train):
    os.makedirs(cropmaskDir_train)

if not os.path.exists(cropsatDir_val):
    os.makedirs(cropsatDir_val)
    
if not os.path.exists(cropmaskDir_val):
    os.makedirs(cropmaskDir_val)

crop_image(trainDir_sat, cropsatDir_train)
crop_image(trainDir_mask, cropmaskDir_train)
crop_image(valDir_sat, cropsatDir_val)
crop_image(valDir_mask, cropmaskDir_val)

In [15]:
from tqdm import tqdm

In [2]:
def encoder_block(inputs, num_filters): 
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3,padding = 'same')(inputs) 
    x = tf.keras.layers.Activation('relu')(x) 

    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'same')(x) 
    x = tf.keras.layers.Activation('relu')(x) 

    # Max Pooling with 2x2 filter 
    x = tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = 2)(x) 
    return x

In [3]:
def decoder_block(inputs, skip_features, num_filters): 
    # Upsampling with 2x2 filter 
    x = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides = 2, padding = 'same')(inputs) 
      
    # Copy and crop the skip features  
    # to match the shape of the upsampled input 
    skip_features = tf.image.resize(skip_features, size = (x.shape[1], x.shape[2])) 
    x = tf.keras.layers.Concatenate()([x, skip_features]) 
      
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'same')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
  
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'same')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
      
    return x

In [4]:
def IoU(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    #y_true_f = np.array(K.flatten(y_true))
    #y_pred_f = np.array(K.flatten(y_pred))
    #y_pred_f[y_pred_f >= 0.5] = 1
    #y_pred_f[y_pred_f < 0.5] = 0
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))

def final_loss(y_true, y_pred):
    loss1 = binary_crossentropy(y_true, y_pred)
    loss2 = 1 - IoU(y_true, y_pred)
    return loss1 + loss2
def unet(pretrained_weights = None,input_size = (256,256,3)):
    
    inputs = Input(input_size)
    
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same')(conv4)
    #drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same')(pool4)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same')(conv5)
    #drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv5))
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(lr = 2e-4), loss = final_loss, metrics = [IoU])
    #model.compile(optimizer = SGD(lr=1e-3, decay=0.0, momentum=0.9, nesterov=True), loss = final_loss, metrics = [IoU])
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [5]:

from keras.preprocessing.image import ImageDataGenerator


# def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "rgb",
#                     mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
#                     flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
#     '''
#     can generate image and mask at the same time
#     use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
#     if you want to visualize the results of generator, set save_to_dir = "your path"
#     '''
#     image_datagen = ImageDataGenerator(**aug_dict)
#     mask_datagen = ImageDataGenerator(**aug_dict)
#     image_generator = image_datagen.flow_from_directory(
#         train_path,
#         classes = [image_folder],
#         class_mode = None,
#         color_mode = image_color_mode,
#         target_size = target_size,
#         batch_size = batch_size,
#         save_to_dir = save_to_dir,
#         save_prefix  = image_save_prefix,
#         seed = seed)
#     mask_generator = mask_datagen.flow_from_directory(
#         train_path,
#         classes = [mask_folder],
#         class_mode = None,
#         color_mode = mask_color_mode,
#         target_size = target_size,
#         batch_size = batch_size,
#         save_to_dir = save_to_dir,
#         save_prefix  = mask_save_prefix,
#         seed = seed)
#     train_generator = zip(image_generator, mask_generator)
#     for (img,mask) in train_generator:
#         #img,mask = adjustData(img,mask,flag_multi_class,num_class)
#         img = img / 255.0
#         mask = mask / 255.0
#         yield (img,mask)


def trainGenerator(batch_size, train_path, image_folder, mask_folder, aug_dict, image_color_mode="rgb",
                    mask_color_mode="grayscale", image_save_prefix="image", mask_save_prefix="mask",
                    flag_multi_class=False, num_class=2, save_to_dir=None, target_size=(256, 256), seed=1):
    '''
    Can generate image and mask at the same time.
    Uses the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same.
    If you want to visualize the results of generator, set save_to_dir = "your path".
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes=[image_folder],
        class_mode=None,
        color_mode=image_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=image_save_prefix,
        seed=seed
    )
    
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes=[mask_folder],
        class_mode=None,
        color_mode=mask_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=mask_save_prefix,
        seed=seed
    )
    
    # Check if generators produce the same number of samples
    if image_generator.batch_size != mask_generator.batch_size:
        raise ValueError("Batch sizes of image_generator and mask_generator must be equal.")

    train_generator = zip(image_generator, mask_generator)
    
    for (img, mask) in train_generator:
        if img.shape[0] != mask.shape[0]:
            raise ValueError("Batch sizes of images and masks do not match.")
        
        # Normalize images and masks
        img = img / 255.0
        mask = mask / 255.0
        
        yield (img, mask)


In [7]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping


In [8]:
data_gen_args = dict(rotation_range=90.,
                    #width_shift_range=0.1,
                    #height_shift_range=0.1,
                    #shear_range=0.1,
                    #zoom_range=0.1,
                    fill_mode='nearest',
                    horizontal_flip=True,
                    vertical_flip=True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor=0.2, patience=3, verbose=0, mode='min', epsilon=1e-4, 
                              cooldown=0, min_lr=1e-6)
visual = TensorBoard(log_dir='./VanillaUNet_log', histogram_freq=0, write_graph=True, write_images=True)
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min')

train_Gene = trainGenerator(1,'./trainData/','cropped_sat','cropped_mask',data_gen_args,save_to_dir = None)
# to be fixed(train-val split)
val_Gene = trainGenerator(1,'./valData/','cropped_sat','cropped_mask',data_gen_args)

In [13]:
model = unet()

C:\Users\subha\miniconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [14]:
model_checkpoint = ModelCheckpoint('U-Net.hdf5', monitor='val_loss',verbose=1, save_best_only=True)
model.fit(train_Gene,steps_per_epoch=50,epochs=10,
                    callbacks=[model_checkpoint, visual, reduce_lr, earlystop], 
                    validation_data=val_Gene, validation_steps=220)#step_per_epoch and validation_steps equals to number of samples divide batchsize

Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 1.7821 - IoU: 0.0410
Epoch 1: val_loss improved from inf to 1.21658, saving model to U-Net.hdf5
50/50 [==============================] - 16s 306ms/step - loss: 1.7821 - IoU: 0.0410 - val_loss: 1.2166 - val_IoU: 0.0389 - lr: 2.0000e-04
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 1.2289 - IoU: 0.0437
Epoch 2: val_loss improved from 1.21658 to 1.15329, saving model to U-Net.hdf5
50/50 [==============================] - 19s 384ms/step - loss: 1.2289 - IoU: 0.0437 - val_loss: 1.1533 - val_IoU: 0.0240 - lr: 2.0000e-04
Epoch 3/10
50/50 [==============================] - ETA: 0s - loss: 1.1619 - IoU: 0.0256
Epoch 3: val_loss improved from 1.15329 to 1.14708, saving model to U-Net.hdf5
50/50 [==============================] - 20s 408ms/step - loss: 1.1619 - IoU: 0.0256 - val_loss: 1.1471 - val_IoU: 0.0141 - lr: 2.0000e-04
Epoch 4/10
50/50 [==============================] - ETA: 0s - loss: 1.1473 - IoU: 0

In [33]:
def predict_z(src, predict_path):
    TEST_SET = os.listdir(src)
    model = unet()
    print('Loading Model weights...')
    model.load_weights('U-Net.hdf5')
    print('Completed!')
    img_h = 256
    img_w = 256
    stride = img_h - 40

    for n in range(len(TEST_SET)):
        path = TEST_SET[n]
        path1 = path[0:-7] + 'mask.png'  # Rename mask
        image = io.imread(os.path.join(src, path))
        h, w, _ = image.shape

        image = image / 255.0
        print('[{}/{}], predicting: {}'.format(n + 1, len(TEST_SET), path))

        mask_whole = np.zeros((h, w, 1), dtype=np.uint8)

        for i in range(0, (h // stride) + 1):
            for j in range(0, (w // stride) + 1):
                h_begin = i * stride
                w_begin = j * stride

                if h_begin + img_h > h:
                    h_begin = h_begin - (h_begin + img_h - h)

                if w_begin + img_w > w:
                    w_begin = w_begin - (w_begin + img_w - w)

                crop = image[h_begin:h_begin + img_h, w_begin:w_begin + img_w, :3]
                ch, cw, _ = crop.shape

                if ch != img_h or cw != img_w:
                    print('Invalid size!')
                    print(i, j, h_begin, w_begin, ch, cw)
                    continue

                crop = np.expand_dims(crop, axis=0)
                pred = model.predict(crop, verbose=2)
                pred = pred.reshape((img_h, img_w, 1)).astype(np.float64)

                # Debugging
                print('Prediction min:', np.min(pred))
                print('Prediction max:', np.max(pred))

                # Thresholding
                pred[pred >= 0.15] = 1
                pred[pred < 0.15] = 0
                pred = pred * 255

                # Update mask
                mask_whole[h_begin:h_begin + img_h, w_begin:w_begin + img_w] = pred

        # Ensure correct shape and dtype
        mask_whole = np.squeeze(mask_whole)
        mask_whole = mask_whole.astype(np.uint8)
        
        # Save mask
        cv2.imwrite(predict_path + path1, mask_whole[0:h, 0:w])
        print('Done!')


In [34]:
fileDir = "./test/"  #test images(1024*1024)
preDir = "./predicted/" #Dir of predict mask
predict_z(fileDir, preDir)  

C:\Users\subha\miniconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Loading Model weights...
Completed!
[1/3], predicting: 58949_sat.jpg
1/1 - 0s - 153ms/epoch - 153ms/step
Prediction min: 0.002804800169542432
Prediction max: 0.3678666353225708
1/1 - 0s - 15ms/epoch - 15ms/step
Prediction min: 0.0027017632964998484
Prediction max: 0.3840187191963196
1/1 - 0s - 17ms/epoch - 17ms/step
Prediction min: 0.0043958681635558605
Prediction max: 0.35241827368736267
1/1 - 0s - 16ms/epoch - 16ms/step
Prediction min: 0.0020188966300338507
Prediction max: 0.34509846568107605
1/1 - 0s - 17ms/epoch - 17ms/step
Prediction min: 0.0019965474493801594
Prediction max: 0.3565346896648407
1/1 - 0s - 16ms/epoch - 16ms/step
Prediction min: 0.0016886458033695817
Prediction max: 0.343956857919693
1/1 - 0s - 18ms/epoch - 18ms/step
Prediction min: 0.001723895431496203
Prediction max: 0.3440151810646057
1/1 - 0s - 16ms/epoch - 16ms/step
Prediction min: 0.0030033860821276903
Prediction max: 0.39681360125541687
1/1 - 0s - 17ms/epoch - 17ms/step
Prediction min: 0.0020015856716781855
P